<a href="https://colab.research.google.com/github/traopia/KGNarrative/blob/master/esperiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

MOUNTPOINT = '/content/gdrive'
Working_Dir = os.path.join(MOUNTPOINT, 'Shareddrives', 'GOT/Building_Narratives_Game_of_Thrones')
drive.mount(MOUNTPOINT)
print(Working_Dir)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/Shareddrives/GOT/Building_Narratives_Game_of_Thrones


In [2]:
!pip install transformers 
!pip install datasets
!pip install pynvml
!pip install evaluate 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import transformers
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import os
import nltk
import torch
import evaluate
import sys
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
from sklearn.model_selection import train_test_split

In [4]:
#! git clone https://github.com/traopia/KGNarrative.git

fatal: destination path 'KGNarrative' already exists and is not an empty directory.


In [5]:
# define utils functions to facilitate gpu 

def check_gpu_availability():
    # Check if CUDA is available
    print(f"Cuda is available: {torch.cuda.is_available()}")

def getting_device(gpu_prefence=True) -> torch.device:
    """
    This function gets the torch device to be used for computations, 
    based on the GPU preference specified by the user.
    """
    
    # If GPU is preferred and available, set device to CUDA
    if gpu_prefence and torch.cuda.is_available():
        device = torch.device('cuda')
    # If GPU is not preferred or not available, set device to CPU
    else: 
        device = torch.device("cpu")
    
    # Print the selected device
    print(f"Selected device: {device}")
    
    # Return the device
    return device

# Define a function to print GPU memory utilization
def print_gpu_utilization():
    # Initialize the PyNVML library
    nvmlInit()
    # Get a handle to the first GPU in the system
    handle = nvmlDeviceGetHandleByIndex(0)
    # Get information about the memory usage on the GPU
    info = nvmlDeviceGetMemoryInfo(handle)
    # Print the GPU memory usage in MB
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

# Define a function to print training summary information
def print_summary(result):
    # Print the total training time in seconds
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    # Print the number of training samples processed per second
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    # Print the GPU memory utilization
    print_gpu_utilization()

In [6]:
# CHECK IF GPU IS UP
check_gpu_availability()



Cuda is available: True


In [7]:
# SAVE THE DEVICE WE ARE WORKING WITH
device = getting_device(gpu_prefence=True)

Selected device: cuda


#### Importing documents


In [8]:
df = pd.read_json('/content/KGNarrative/generated_dataset/Instances/DWIE_train.json')


In [9]:
df.head(10)

,story,Instances Knowledge Graph,Types Knowledge Graph,Subclass Knowledge Graph
0,Multi-lingual in the World Wide Web Profession...,Personal Translator - product_of - Linguatec |...,Personal Translator - type - entity | Personal...,misc - subclass_of - entity | product - subcla...
1,German Know-How For China's Energy Sector Ener...,Shangdong Linuo Paradigma - based_in0 - German...,China - type - entity | China - type - gpe | C...,gpe - subclass_of - location | gpe0 - subclass...
2,Trash crisis forces Lebanon's environmental aw...,Kassem Kazak - citizen_of - Lebanon | Kassem K...,Kareem Chehayeb - type - entity | Kareem Cheha...,journalist - subclass_of - per | manager - sub...
3,"Iran calls for end to Saudi air campaign, as U...",Houthi - based_in0 - Yemen | Houthi - based_in...,Medecins Sans Frontieres - type - entity | Med...,ngo - subclass_of - org | gpe - subclass_of - ...
4,Ai Weiwei Drifting Ai Weiwei: Uncomfortable cr...,DW (Deutsch+) - based_in0 - Germany | Ai Weiwe...,DW (Deutsch+) - type - entity | DW (Deutsch+) ...,media - subclass_of - org | activist - subclas...
5,"Journal Interview with Günter Nooke, The Chanc...",Günter Nooke - agent_of - Germany | Günter Noo...,Günter Nooke - type - entity | Günter Nooke - ...,gov_per - subclass_of - per | location - subcl...
6,Turkey summons German envoy for second time in...,Martin Erdmann - agent_of - Germany | Martin E...,European Union - type - entity | European Unio...,igo - subclass_of - org | so - subclass_of - i...
7,Medvedev seeks investment at Davos economic fo...,Dmitry Medvedev - agent_of - Russia | Dmitry M...,Dmitry Medvedev - type - entity | Dmitry Medve...,head_of_gov - subclass_of - politician | polit...
8,Ex-CIA fugitive Robert Seldon Lady detained in...,Osama Moustafa Hassan Nasr - citizen_of - Egyp...,Osama Moustafa Hassan Nasr - type - clergy | O...,offender - subclass_of - per | agency - subcla...
9,"Nearly 10,000 migrants rescued on Mediterranea...",Rome - in0 - Italy | Rome - in0-x - Italian | ...,Strait of Sicily - type - entity | Strait of S...,location - subclass_of - entity | waterbody - ...


#### News classification


In [10]:
model_nm1 = 'abhishek/autonlp-bbc-news-classification-37229289'

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

# Load the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_nm1)
model = AutoModelForSequenceClassification.from_pretrained(model_nm1).to(device)

In [12]:
print(print_gpu_utilization())

GPU memory occupied: 2119 MB.
None


In [13]:
# Create a list to hold the predicted labels
predicted_type_of_news= []
type(predicted_type_of_news)


list

In [14]:


# Load the configuration of the model
config = AutoConfig.from_pretrained("abhishek/autonlp-bbc-news-classification-37229289")
labels = config.label2id.keys()

In [15]:
# Loop over each story in the DataFrame
max_length = tokenizer.model_max_length
#TODO: make it better (bc max input of those models is 512)
for story in df["story"]:
    # Encode the story text using the tokenizer
    inputs = tokenizer.encode(story, max_length= max_length, return_offsets_mapping=False, stride=0, return_tensors='pt').to(device)
    
    # Make a prediction using the model
    outputs = model(inputs)

    # Get the predicted label
    predicted_label_id = outputs.logits.argmax().item()
    
    predicted_label = list(labels)[predicted_label_id]

    # Add the predicted label to the list

    # Add the predicted label to the list
    predicted_type_of_news.append(predicted_label)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [16]:
# Add the predicted labels to the DataFrame
df["predicted_label1"] = predicted_type_of_news
df.head(10)

,story,Instances Knowledge Graph,Types Knowledge Graph,Subclass Knowledge Graph,predicted_label1
0,Multi-lingual in the World Wide Web Profession...,Personal Translator - product_of - Linguatec |...,Personal Translator - type - entity | Personal...,misc - subclass_of - entity | product - subcla...,tech
1,German Know-How For China's Energy Sector Ener...,Shangdong Linuo Paradigma - based_in0 - German...,China - type - entity | China - type - gpe | C...,gpe - subclass_of - location | gpe0 - subclass...,business
2,Trash crisis forces Lebanon's environmental aw...,Kassem Kazak - citizen_of - Lebanon | Kassem K...,Kareem Chehayeb - type - entity | Kareem Cheha...,journalist - subclass_of - per | manager - sub...,tech
3,"Iran calls for end to Saudi air campaign, as U...",Houthi - based_in0 - Yemen | Houthi - based_in...,Medecins Sans Frontieres - type - entity | Med...,ngo - subclass_of - org | gpe - subclass_of - ...,business
4,Ai Weiwei Drifting Ai Weiwei: Uncomfortable cr...,DW (Deutsch+) - based_in0 - Germany | Ai Weiwe...,DW (Deutsch+) - type - entity | DW (Deutsch+) ...,media - subclass_of - org | activist - subclas...,entertainment
5,"Journal Interview with Günter Nooke, The Chanc...",Günter Nooke - agent_of - Germany | Günter Noo...,Günter Nooke - type - entity | Günter Nooke - ...,gov_per - subclass_of - per | location - subcl...,politics
6,Turkey summons German envoy for second time in...,Martin Erdmann - agent_of - Germany | Martin E...,European Union - type - entity | European Unio...,igo - subclass_of - org | so - subclass_of - i...,politics
7,Medvedev seeks investment at Davos economic fo...,Dmitry Medvedev - agent_of - Russia | Dmitry M...,Dmitry Medvedev - type - entity | Dmitry Medve...,head_of_gov - subclass_of - politician | polit...,business
8,Ex-CIA fugitive Robert Seldon Lady detained in...,Osama Moustafa Hassan Nasr - citizen_of - Egyp...,Osama Moustafa Hassan Nasr - type - clergy | O...,offender - subclass_of - per | agency - subcla...,business
9,"Nearly 10,000 migrants rescued on Mediterranea...",Rome - in0 - Italy | Rome - in0-x - Italian | ...,Strait of Sicily - type - entity | Strait of S...,location - subclass_of - entity | waterbody - ...,business


In [18]:
df.to_csv('DWIE_train_topic_news.csv')

#### 